# Flæðisverkefni í verkefnatíma 8 í Greiningu Reiknirita í HÍ

Setja þarf upp Google OR-Tools pakkann með eftirfarandi skipun. Athugið að þið þurfið að tengjast umhverfinu aftur eftir að þið hafið sett pakkann upp. Það mun birtast takki sem gerir ykkur kleyft að gera það.

In [1]:
!pip install ortools

Defaulting to user installation because normal site-packages is not writeable


Fyrst búum við til netið sem við ætlum að vinna með í verkefninu. Ekki þarf að breyta eftirfarandi kóðabút. Allir vinna með sama netið í þessu verkefni.

In [2]:
# Búum til netið, það þarf ekki að breyta kóða í þessari sellu
import numpy as np
np.random.seed(42)

n = 400
p = 0.05

partition_size = 50
partition_segments = [(x,x+partition_size) for x in range(1,n-partition_size,partition_size)]

start_nodes = [] # Upphafshnútur leggjar
end_nodes = [] # Endahnútur leggjar
capacities = [] # Rými 
for n_min, n_max in partition_segments:
  for i in range(n_min,n_max):
    for j in range(n_min+partition_size,n_max+partition_size):
      if i != j and np.random.random() < p:
        start_nodes.append(i)
        end_nodes.append(j)
        capacities.append(np.random.randint(1,10))

# Leggir frá upphafshnút
for i in range(1,1+partition_size):
  start_nodes.append(0)
  end_nodes.append(i)
  capacities.append(np.random.randint(20,40))

# Leggir að endahnút
for i in range(n+1-partition_size,n+1):
  start_nodes.append(i)
  end_nodes.append(401)
  capacities.append(np.random.randint(20,40))

Verkefnið byrjar í raun hér, þið þurfið að reikna hámarksflæðið í netinu og segja til um hvort það sé einstakt eða ekki.

In [3]:
from __future__ import print_function
from ortools.graph.python import max_flow

# Fáum solver sem getur leyst flæðisverkefni
smf = max_flow.SimpleMaxFlow()
# Byggjum netið í einni skipun, all_arcs inniheldur alla leggi
all_arcs = smf.add_arcs_with_capacity(start_nodes, end_nodes, capacities)

# Hámarksflæði
Hvert er hámarksflæðið í netinu frá hnút 0 að hnút 401? Reiknaðu út hámarksflæðið með því að nota OR-Tools pakkann. Þú getur fundið leiðbeiningar hér: https://developers.google.com/optimization/flow/maxflow

In [4]:
max0to401 = smf.solve(0, 401)
old_flow = smf.optimal_flow()
print(f'hámarksflæði frá 0 að 401 er {old_flow}')

hámarksflæði frá 0 að 401 er 283


# Er flæðið einstakt?
Á heimaverkefni 7 var dæmi þar sem kanna átti hvort flæði væri einstakt eða ekki. Prófaðu að kanna hvort það sé raunin hér eða ekki.

Fyrir sérhvern legg sem er mettaður í hámarksflæðinu skaltu prófa eftirfarandi:

1. lækkaðu rými leggjarins um einn
2. reiknaðu aftur hámarksflæði og kannaðu hvort það hafi sama gildi og hámarksflæðið sem þú fannst áður
3. hækkaðu rými leggjarins aftur um einn áður en þú prófar næsta legg

Ef þú færð sama hámarksflæði og áður fyrir einhvern legg þá getur þú ályktað að flæðið sé ekki einstakt. Ef þú lendir í því að hámarksflæðið lækkar skaltu prenta að leggurinn sé takmarkandi að neðan.

Þú getur notað `smf.SetArcCapacity(e,x)` til að láta legginn `e` fá rýmið `x`.

In [12]:
solution_flows = smf.flows(all_arcs)

for arc, flow, capacity in zip(all_arcs, solution_flows, capacities):
    if flow == capacity:
        smf.set_arc_capacity(arc, capacity - 1)
        new_flow = smf.solve(0, 401)

        if old_flow == smf.optimal_flow():
            print(f"flæði er ekki einstakt, fannst við breytingu á legg {smf.tail(arc)} -> {smf.head(arc)} þ.e. vegur nr {arc}")
            smf.set_arc_capacity(arc, capacity)
            break
        smf.set_arc_capacity(arc, capacity)


flæði er ekki einstakt, fannst við breytingu á legg 7 -> 65 þ.e. vegur nr 21
